In [ ]:
#Instalacion de librerias
!pip install SpeechRecognition
!pip install nltk
!pip install googletrans==4.0.0-rc1
!pip install pyttsx3
!pip install spacy-langdetect
!pip install pyaudio
!pip install pandas numpy matplotlib
!pip install spacy
!pip install keyboard

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
#Importacion de Bibliotecas

#Grabar la voz
import speech_recognition as sr
import pandas as pd
#Mostrar letras diseño
from IPython.display import display as display_function, HTML as html_print, Markdown as display_markdown
#Traducir el texto o voz
from googletrans import Translator
#Convertir texto a voz
import pyttsx3
#PLN
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector
from nltk.corpus import stopwords

In [ ]:
#Funciones para inicializar Spacy
def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load('en_core_web_sm')
try:
    Language.factory("language_detector", func=get_lang_detector)
except Exception as e:
    next
    
nlp.add_pipe('language_detector', last=True)

In [ ]:
#Funcion para intersectar listas
def Intersection(lst1, lst2):
    return set(lst1).intersection(lst2)

In [ ]:
#Funciones para imprimir mensajes a colores
def cstr(s, color='black', size = '50'):
  return "<text style='color:{};font-size:{}px'>{}</text>".format(color,size, s)

def print_color(t):
  display(html_print(' '.join([cstr(ti, color=ci, size=si) for ti,ci,si in t])))

In [ ]:
#Inicializar variables
r = sr.Recognizer()
r.energy_threshold = 300
translator = Translator()
converter = pyttsx3.init()
converter.setProperty('rate', 150)
converter.setProperty('volume', 1)
converter.setProperty('voice', 'com.apple.speech.synthesis.voice.juan')
lista_fr = stopwords.words('french')
lista_en = stopwords.words('english')
lista_es = stopwords.words('spanish')
lista_pt = stopwords.words('portuguese')

In [ ]:
# Funcion para reconocer el sonido
def reconocer(sonido):
    lista = ['fr-FR', 'pt-BR', 'es-ES', 'en-US']
    fr, pt, es, en = 0, 0, 0, 0
    df = pd.DataFrame(columns=['query', 'idioma', 'valor', 'palabras', 'stopwords', 'idioma_final', 'match', 'nota'])
    idioma_final = 'Ninguno'

    for elemento in lista:
        try:
            query = r.recognize_google(sonido, language=elemento)
            if query.lower() == "detener traducción":
                print("Deteniendo la traducción...")
                return "STOP"
            
            valor = nlp(query)._.language.get('score')
            idioma = nlp(query)._.language.get('language')
            palabras = len(query.split())
            stopwords = 0
            if idioma == 'fr': stopwords, fr, idioma_final = len(Intersection(query.split(), lista_fr)), (fr+1), 'Frances'
            if idioma == 'pt': stopwords, pt, idioma_final = len(Intersection(query.split(), lista_pt)), (pt+1), 'Portugues'
            if idioma == 'es': stopwords, es, idioma_final = len(Intersection(query.split(), lista_es)), (es+1), 'Español'
            if idioma == 'en': stopwords, en, idioma_final = len(Intersection(query.split(), lista_en)), (en+1), 'Ingles'
            if idioma == elemento[0:2]: df.loc[len(df)] = [query, idioma, valor, palabras, stopwords, idioma_final, 0, 0.0]
        except Exception as e:
            print(e)
            continue

    for index, row in df.iterrows():
        if row['idioma'] == 'fr': df.at[index, 'match'] = fr
        if row['idioma'] == 'pt': df.at[index, 'match'] = pt
        if row['idioma'] == 'es': df.at[index, 'match'] = es
        if row['idioma'] == 'en': df.at[index, 'match'] = en

    df['nota'] = (df['palabras'] + df['stopwords']) * df['valor'] * df['match']
    resultado = df.loc[df['nota'].idxmax()] if not df.empty else None

    if resultado is not None and len(df) > 0:
        print_color(((resultado['idioma_final'] + ' detectado', 'blue', '40'),))
        return resultado['query'], resultado['idioma_final']
    else:
        print("No se reconoció ningún idioma.")
        return 'Nada'

In [ ]:
# Funcion para escuchar el microfono (modificada para usarse en la interfaz gráfica)
def takecommand():
    while True:
        with sr.Microphone(device_index=0) as source:
            r.adjust_for_ambient_noise(source, duration=0.5)
            try:
                audio = r.listen(source, timeout=4)
            except sr.WaitTimeoutError:
                print("No se detectó audio en 4 segundos.")
                continue
        
        try:
            mensaje = reconocer(audio)
            if mensaje == "STOP":
                break
            return mensaje
        except Exception as e:
            print(e)
            continue


In [ ]:
# Crear la interfaz gráfica
def start_listening():
    texto_escuchado, idioma_detectado = takecommand()
    if texto_escuchado != 'Nada' and texto_escuchado != "STOP":
        # Traducir el mensaje a español
        idioma_salida = idioma_selector.get()
        idiomas_nombre = {
            'es': 'Español',
            'en': 'Inglés',
            'fr': 'Francés',
            'pt': 'Portugués'
        }
        text_to_translate = translator.translate(texto_escuchado, dest=idioma_salida)
        translated_text = text_to_translate.text
        # Mostrar en la interfaz el texto (sin borrar el contenido anterior)
        text_output.insert(tk.END, f"La persona dijo ({ idioma_detectado}): {texto_escuchado}\n")
        text_output.insert(tk.END, f"Traducción ({idiomas_nombre[idioma_salida]}): {translated_text}\n")
        converter.say(translated_text)
        converter.runAndWait()
    elif texto_escuchado == "STOP":
        root.quit()

def stop_listening():
    global ejecutando
    ejecutando = False
    text_output.delete(1.0, tk.END)  # Limpiar el área de texto


In [ ]:
import tkinter as tk
from tkinter import scrolledtext
# Configurar la ventana principal de Tkinter
root = tk.Tk()
root.title("Reconocimiento de Voz y Traducción")
root.geometry("500x300")  # Tamaño fijo de la ventana
root.resizable(False, False)  # No permitir que la ventana se expanda

# Colores
colores = {
    'azul_claro': '#a2d2ff',
    'azul_pastel': '#bde0fe',
    'rosa_claro': '#ffafcc',
    'rosa_pastel': '#ffc8dd',
    'lavanda': '#cdb4db',
    'blanco': '#ffffff'
}

# Configuración de la ventana
root.configure(bg=colores['azul_pastel'])
idioma_selector_label = tk.Label(root, text="Traductor Multilingüe", font=("Arial", 12), bg=colores['azul_pastel'], fg='black')
idioma_selector_label.pack(side=tk.TOP, padx=10, pady=10)

# Área de texto para mostrar lo que se escucha y la traducción
text_output = scrolledtext.ScrolledText(root, width=50, height=10, wrap=tk.WORD, font=("Arial", 12), bg=colores['blanco'], fg='black', relief="solid", bd=2)
text_output.pack(padx=10, pady=10)

# Crear el selector de idioma y botones en la barra inferior
frame_inferior = tk.Frame(root, bg=colores['azul_pastel'], pady=10)

# Crear el selector de idioma
idiomas_disponibles = ['es', 'en', 'fr', 'pt']
idioma_selector_label = tk.Label(frame_inferior, text="Selecciona el idioma de traducción:", font=("Arial", 12), bg=colores['azul_pastel'], fg='black')


idioma_selector = ttk.Combobox(frame_inferior, values=idiomas_disponibles, font=("Arial", 12),state="readonly")
idioma_selector.set('es')  # Idioma por defecto (Español)
idioma_selector.pack(side=tk.LEFT, padx=10)

# Botones
btn_start = tk.Button(frame_inferior, text="Hablar", width=15, font=("Arial", 12), command=start_listening, bg=colores['rosa_claro'], fg='black', relief="solid", bd=2)
btn_start.pack(side=tk.LEFT, padx=10)

btn_stop = tk.Button(frame_inferior, text="Limpiar", width=15, font=("Arial", 12), command=stop_listening, bg=colores['rosa_pastel'], fg='black', relief="solid", bd=2)
btn_stop.pack(side=tk.LEFT, padx=10)

frame_inferior.pack(side=tk.BOTTOM, fill=tk.X)  # Barra inferior que contiene el selector y los botones

# Ejecutar la interfaz gráfica
root.mainloop()